In [9]:
import sys
sys.path.append('/users/nfoster3/data/nfoster3/two_bit_bananas')
sys.path.append('/users/nfoster3/data/nfoster3/two_bit_bananas/simple_test')

from torch.utils.data import DataLoader
from FakeData import CorrelatedDataset
from tqdm import tqdm
import torch

from SimpleModel import AnnealedModel

# Create a CorrelatedDataset object with noise=0.2
correlated_dataset = CorrelatedDataset(num_samples=1000)

# Create a DataLoader with batch size 32 and shuffle the data with 10% set asside for testing
dataloader = DataLoader(correlated_dataset, batch_size=32, shuffle=True, drop_last=True)

In [10]:
def train(model, epochs, dataloader, lr=1e-4, final_temperature=200.0, device = 'cuda:0'):
    # Set the model to training mode
    model.train()
    model = model.to(device)
    # Create an optimizer
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    
    temp_increase = (final_temperature -1.0) ** (1.0 / (epochs -1))
    # Create a progress bar
    progress_bar = tqdm(range(epochs))
    
    # Loop over the epochs
    for epoch in progress_bar:
        # Loop over the data
        for data, labels in tqdm(dataloader):
            # Move the data to the device
            data = data.to(device)
            labels = labels.to(device)
            # Zero out the gradients
            optimizer.zero_grad()
            
            # 1-Hot encode the labels
            labels = torch.nn.functional.one_hot(labels, num_classes=2).float()
            loss = torch.nn.functional.binary_cross_entropy_with_logits(model(data), labels)

            # Compute Training Accuracy
            predictions = torch.sigmoid(model(data))
            predictions = torch.round(predictions)
            accuracy = torch.sum(predictions == labels) / labels.shape[0]
            
            # Backpropagate the loss
            loss.backward()
            
            # Take a step
            optimizer.step()

            # Update the progress bar
            progress_bar.set_description(f'Epoch {epoch} Loss {loss.item():.4f} Temperature {model.temperature:.4f} Accuracy {accuracy:.4f}')

            # Increase the temperature
        model.set_temperature(model.temperature * temp_increase)


In [11]:
model = AnnealedModel(
    input_size=10,
    output_size=2,
    hidden_size=10,
    hidden_layers=0,
    bias=True
)

In [12]:
train(model, 10, dataloader, lr=1e-4, final_temperature=200.0)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 173.66it/s]
Epoch 9 Loss 0.6945 Temperature 199.0001 Accuracy 0.9375: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.47it/s]


In [13]:
model.eval()

In [14]:
test_dataset = CorrelatedDataset(num_samples=100)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True, drop_last=True)

In [15]:
def test(model, test_dataloader):
    model.eval()
    model = model.to('cuda:0')
    total = 0
    correct = 0
    for data, labels in test_dataloader:
        data = data.to('cuda:0')
        labels = labels.to('cuda:0')
        predictions = torch.nn.functional.softmax(model(data), dim=1)
        predictions = torch.argmax(predictions, dim=1)
        correct += torch.sum(predictions == labels)
        total += labels.numel()
    return correct / total

In [18]:
result = test(model, test_dataloader)
print(f"Test Accuracy: {result:.4f}")
for i, layer in enumerate(model.layers):
    print(f"Layer {i}")
    print(layer.weight)
    print(layer.bias)

Test Accuracy: 0.5000
Layer 0
tensor([[-1., -1., -1., -1., -1.,  2.,  2.,  2.,  2.,  2.],
        [-1., -1., -1., -1., -1.,  2.,  2.,  2.,  2.,  2.],
        [-1., -1., -1., -1., -1.,  2.,  2.,  2.,  2.,  2.],
        [-1., -1., -1., -1., -1.,  2.,  2.,  2.,  2.,  2.],
        [-1., -1., -1., -1., -1.,  2.,  2.,  2.,  2.,  2.],
        [-1., -1., -1., -1., -1.,  2.,  2.,  2.,  2.,  2.],
        [-1., -1., -1., -1., -1.,  2.,  2.,  2.,  2.,  2.],
        [-1., -1., -1., -1., -1.,  2.,  2.,  2.,  2.,  2.],
        [-1., -1., -1., -1., -1.,  2.,  2.,  2.,  2.,  2.],
        [-1., -1., -1., -1., -1.,  2.,  2.,  2.,  2.,  2.]], device='cuda:0')
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.], device='cuda:0')
Layer 1
tensor([[-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.]], device='cuda:0')
tensor([-1., -1.], device='cuda:0')
